# Data Engineer Challenge

## Problem Statement

You’ve been tasked to design a solution in a way that would aggregate the three data sources into one whilst providing a unique view of the consolidated data. Please include the technologies you would use to build it. !

## Psuedo code

1.  The design accompanied by an explanation for why you chose the specific technology.
2.	An executable prototype of the pipeline managing the loading + aggregation of the data providing a unique view of the output. Ensuring the quality of the data and the code.
3.	Discuss quickly the scalability and the deployment of the solution.

## Steps

I have put forward design from three perspectives taking a more generalistic approach considering the broadness of various ways of implementing the solution. We have three sources here:

The database.csv has the user information - user_id is the primary key. This is a master data that needs to be loaded via batch process and ideally maintained in NOSQL table like HBASE, DYNAMODB for ease of access by the real-time streaming engine.

The third_party_http_data.json is a JSON formatted data. Ideally in real-world setting this must be a client REST API request that fetches this data in real-time. The email-id is the primary key and based on that this api fetches the company profile and social presence related information for the customer. 
Real-world scenario implementation - Http Client implemented with Circuit Breaker Pattern for resilence and fast failure to avoid flooding http requests in case of HTTP Server unavailability.

The message_bus.json is the real-time data in JSON format. This data must usually be produced to pub-sub management platform like Kafka or Kinesis from sources. The consumers can subscribe to the topic to consume the data in real-time. The spark-streaming can be used to process the data and enrich with user information and social presence information. Upon processing, the data can be persisted in hdfs in any format (orc, parquet etc.,) for ease of consumption from BI Tools like tableau, quicksight or for any real-time reporting platform.

The detailed potential solution design is present in the SafetyCultureSolutionDesign.pptx attached.

## Loading Datasets

### Message HTTP
A 3rd party service that we use to enrich data which we extract via a HTTP request (3rd_party_data.json)!

### Message BUS - Real Time data - Message driven
A messaging system where we receive real time data from multiple sources (message_bus.json)!

### Databases 
Multiple types of Databases, SQL and NoSQL. These databases also provide data that we can read and maintain. (database.csv)!

### Application

The below code requires the paths to be passed while instantiating the class. The parameters are:

userInfo - The full path to database.csv
thirdPartyInfo - The full path to 3rd_party_data.json
messageBusInfo - The directory where message_bus.json file is present
checkPointLocation - The path to checkpoint folder
outputPath - The path to output folder

Executing the below code will set up the class. The application instantiation code below that will pass the above parameter and start the consumer process. The final output orc file will be generated in the output path.


In [60]:
## Imports
from pyspark import SparkContext as sc
from pyspark.sql import SparkSession, Row
from pyspark.sql.functions import *
from pyspark.sql.types import *

# SparkSession and Log4j initialization
spark = SparkSession.builder.appName("SafetyCultureDataLake").getOrCreate()
log4jLogger = spark._jvm.org.apache.log4j
log = log4jLogger.LogManager.getLogger(__name__)

# Message bus data load
class MessageBusStream:
    def __init__(self,userInfo,thirdPartyInfo,messageBusInfo,checkPointLocation,outputPath):
        userInfo=userInfo
        thirdPartyInfo=thirdPartyInfo
        messageBusInfo=messageBusInfo
        checkPointLocation=checkPointLocation
        outputPath=outputPath

    # userInfoDB
    def loadUserInfo(self,filePath, format = "csv"):
        log.info(f"loading User Info from ${filePath}")
        userInfoDB = spark.read.format(format).options(header="true", inferSchema="true").load(filePath)
        log.info(f"Finished loading User Info from ${filePath}")
        return userInfoDB
        # produceDataToStream()

    def thirdPartySocialInfo(self,filePath, format = "json"):
        thirdPartyHttpFields = [StructField("user_email", StringType(), True),StructField("industry", StringType(), True),StructField("region", StringType(), True),StructField("compay_profile", StringType(), True),StructField("social_presence", StringType(), True)]
        thirdPartyHttpSchema = StructType(thirdPartyHttpFields)
        thirdPartyHttpData = spark.read.format(format).schema(thirdPartyHttpSchema).load(filePath)
        return thirdPartyHttpData

    def produceMessageBusStream(self,filePath):
        messageBusFields = [StructField("user_id", StringType(), True),StructField("checklist_id", StringType(), True),StructField("checklist_name", StringType(), True),StructField("items", ArrayType(MapType(StringType(), StringType()), True), True)]
        messageBusSchema = StructType(messageBusFields)
        messageBusData = spark.readStream.format("json").schema(messageBusSchema).load(filePath)
        return messageBusData

    def enrichMessageBusData(self,userInfo, thirdPartyInfo):
    #   Load Lookup Data
        userInfoDB = loadUserInfo(filePath=userInfo)
        thirdPartyHttpData = thirdPartySocialInfo(thirdPartyInfo)
        messageBusData = produceMessageBusStream(messageBusInfo)
        messageBusSelectedCols = messageBusData.selectExpr("user_id","checklist_id","checklist_name","explode(items) as item_details").withColumnRenamed("col", "item_details").withColumnRenamed("user_id","message_bus_user_id")
        messageBusDataWithUserInfoDF = messageBusSelectedCols.join(userInfoDB,messageBusSelectedCols.message_bus_user_id == userInfoDB.user_id,"inner").drop("user_id")
        messageBusDataWithUserInfoDF.createOrReplaceTempView("messageBusDataWithUserInfoView")
        messageBusDataWithUserSocialInfo = messageBusDataWithUserInfoDF.join(thirdPartyHttpData, messageBusDataWithUserInfoDF.email == thirdPartyHttpData.user_email, "inner")
        return messageBusDataWithUserSocialInfo

    def consumerStart(self):
        messageBusDataWithUserSocialInfo = enrichMessageBusData(userInfo=userInfo, thirdPartyInfo=thirdPartyInfo)
        queryUserSocialInfo = messageBusDataWithUserSocialInfo.writeStream.format("orc").option("checkpointLocation", checkPointLocation).option("header", True).option("path", outputPath).start()
        queryUserSocialInfo.awaitTermination()
    
    

## Application 

Below code triggers the streaming application and produces output in the output folder. It monitors for new file in the messageBusInfo path and produces output in the output path.

In [61]:
## Initialize the paths to the file:
userInfo="<path to database.csv file>/database.csv"
thirdPartyInfo="path to 3rd party data.json file/3rd_party_data.json"
messageBusInfo="directory where the message_bus.json file will be placed"
checkPointLocation="<random local directory in drive for checkpointing>"
outputPath="<path where the output file will be produced>"

# Instantiate the application
app = MessageBusStream(userInfo,thirdPartyInfo,messageBusInfo,checkPointLocation,outputPath)

# Start the application
app.consumerStart()

KeyboardInterrupt: 

### Results

### Note: The output above is due to keyboard interruption to stop the streaming job as it continuously runs !!

The results are captured in orc format. Spark has inbuilt utility to read the orc file into dataframe and this was used for this exercise. Ideally, this can be consumed using BI tools. Please replace the name of the file in the below code to get the output.

### Note: The final_output.orc file attached with the results has the output that i got. Kindly place that file in local drive and provide the path below to that file to see the output.

In [63]:
result = spark.read.orc("<directory to the output orc file>/final_output.orc")
result.where(col("message_bus_user_id") == "ffbf731b-897f-465e-b4c7-2f5efa54989d").collect()

[Row(message_bus_user_id='ffbf731b-897f-465e-b4c7-2f5efa54989d', checklist_id='c9d557ee-a580-46af-91e6-60fb8ec9bc9a', checklist_name='Checklist No 243', item_details={'item_title': 'item number 0', 'item_id': 'e2953563-3c29-451a-8611-b775e4bb3190', 'item_value': '0'}, email='gumpish@yahoo.ca', name='Ayana Davidson', address='Address 16', country='Australia', city='Sydney', postcode=2000, company_name='Company_16', user_email='gumpish@yahoo.ca', industry='industry_5', region='region_1', compay_profile='profile', social_presence='text_text'),
 Row(message_bus_user_id='ffbf731b-897f-465e-b4c7-2f5efa54989d', checklist_id='35041ec8-58df-4c0e-b6f4-1d76cc7de350', checklist_name='Checklist No 561', item_details={'item_title': 'item number 11', 'item_id': '19fb9ab5-5e81-4159-8840-269438ea4f50', 'item_value': '11'}, email='gumpish@yahoo.ca', name='Ayana Davidson', address='Address 16', country='Australia', city='Sydney', postcode=2000, company_name='Company_16', user_email='gumpish@yahoo.ca', in